In [ ]:
using QuantumCollocation
using CairoMakie

ops = Options()
ops.recalc_y = "yes"
ops.recalc_y_feas_tol = 0.001

# Robust control
-----

Robust control is about being insensitive to *known unknowns*. For example, we might *know* we did a bad job characterizing our qubit frequency. This means we have an operator $\zeta Z$ hanging around in our model. We *know* $Z$ is where the problem shows up, but $\zeta$ is *unknown*--we aren't sure how bad things are. Specifically,
\begin{align}
    &H(t) = \zeta Z + \frac{1}{2} a(t) X \\
    &H_\text{control}(t) 
    = \frac{1}{2} a(t) X
\end{align}


In [ ]:
H_Z = GATES[:Z]
H_X = GATES[:X]

# X gate
U_goal = [0 1; 1 0]

ζ = 0.05
system = QuantumSystem(ζ * H_Z, [H_X])
control_system = QuantumSystem([H_X])

## Set up
The true dynamics are:
\begin{equation}
    i \frac{d}{dt} U(t) = H(t) U(t).
\end{equation}

The control model dynamics are:
\begin{equation}
    i \frac{d}{dt} U_\text{control}(t) = H_\text{control}(t) U_\text{control}(t).
\end{equation}

There is a discrepancy $H(t) - H_\text{control}(t) \underset{\text{e.g.}} = \zeta Z$.

## Cost function

The usual optimization problem to achieve a gate G involves minimizing infidelity,
\begin{equation}
    \text{infidelity}(U) = 1 - |\text{Tr}(U^\dagger G)|^2
\end{equation}

But we don't have access to the true dynamics, so
\begin{equation}
    \text{infidelity}(U) = 1 - |\text{Tr}(U^\dagger U_\text{control} U^\dagger_\text{control} G)|^2.
\end{equation}

If we do a good job optimizing for the gate, $U^\dagger_\text{control} \approx G$. What's leftover is
\begin{equation}
    \text{leftover}(U) = 1 - |\text{Tr}(U^\dagger U_\text{control})|^2.
\end{equation}

In [ ]:
T = 50
Δt = 0.2

prob = UnitarySmoothPulseProblem(
    control_system, U_goal, T, Δt,
    hessian_approximation=true, ipopt_options=ops
)

In [ ]:
solve!(prob; max_iter=200)
println("\nInfidelity: ", 1 - unitary_fidelity(prob))

Plot the controls

In [ ]:
f = Figure(resolution = (800, 400))
ax = f[1, 1] = Axis(f, xlabel = "Time", ylabel = "Control amplitude")
for row in eachrow(prob.trajectory[:a])
    lines!(ax, Δt .* (1:T), row, color=:blue, linewidth=3)
    band!(ax, Δt .* (1:T), zeros(size(row)), row, color = (:blue, 0.1))
end
f

Plot the states

In [ ]:
# Check the control system
Ũ⃗_rollout = unitary_rollout(prob.trajectory, control_system)

f = Figure(resolution = (800, 400))
ax = f[1, 1] = Axis(f, xlabel = "Time", ylabel = "Unitary component")
for (row, is_real) in zip(eachrow(Ũ⃗_rollout), operator_to_iso_vec(ones(2, 2)))
    lines!(ax, Δt .* (1:T), row, linewidth=5, linestyle = Bool(is_real) ? :solid : :dash)
end
f

In [ ]:
# What does the drift do to the control result?
Ũ⃗_rollout = unitary_rollout(prob.trajectory, system)

f = Figure(resolution = (800, 400))
ax = f[1, 1] = Axis(f, xlabel = "Time", ylabel = "Unitary component")
for (row, is_real) in zip(eachrow(Ũ⃗_rollout), operator_to_iso_vec(ones(2, 2)))
    lines!(ax, Δt .* (1:T), row, linewidth=5, linestyle = Bool(is_real) ? :solid : :dash)
end
f

## Addressing what's leftover

The leftover bit is
\begin{equation}
    \text{leftover}(U) = 1 - |\text{Tr}(U^\dagger U_\text{control})|^2.
\end{equation}

We can compute the Hamiltonian for $U^\dagger U_\text{control}$ by taking the derivative. The result is
\begin{equation}
    i\frac{d}{dt}(U^\dagger U_\text{control}) = \underbrace{U^\dag (H - H_\text{control})U}_\text{new Hamiltonian} (U^\dagger U_\text{control})
\end{equation}

This new Hamiltonian is useful for doing perturbation theory on the leftovers. The result is a new cost that is small if we are insensitive to our known unknown,
\begin{equation}
     \text{leftover}(U) \approx \text{robustness}(U_1, U_2, \dots U_T) = |\text{Tr} (\sum\nolimits_{t=1}^{T} U_t^\dag (H - H_\text{control})U_t)^2|
\end{equation}

In [ ]:
robust_prob = UnitaryRobustnessProblem(
    H_Z, prob,
    final_fidelity=0.9999, ipopt_options=ops #objective=DefaultObjective()
)

In [ ]:
solve!(robust_prob; max_iter=200)
println("\nInfidelity: ", 1 - unitary_fidelity(prob))

Plot the controls

In [ ]:
f = Figure(resolution = (800, 400))
ax = f[1, 1] = Axis(f, xlabel = "Time", ylabel = "Control amplitude")
for row in eachrow(robust_prob.trajectory[:a])
    lines!(ax, Δt .* (1:T), row, color=:blue, linewidth=3)
    band!(ax, Δt .* (1:T), zeros(size(row)), row, color = (:blue, 0.1))
end
f

It's a piecewise constant solution with respect to acceleration.

In [ ]:
f = Figure(resolution = (800, 400))
ax = f[1, 1] = Axis(f, xlabel = "Time", ylabel = "Acceleration amplitude")
for row in eachrow(robust_prob.trajectory[:dda])
    stairs!(ax, Δt .* (1:T), row, color=:red, linewidth=3, step=:center)
    band!(ax, Δt .* (1:T), zeros(size(row)), row, color = (:red, 0.1))
end
f

Plot the states

In [ ]:
# System without drift
f = Figure(resolution = (800, 600))
for (i, problem) in enumerate([prob, robust_prob])
    Ũ⃗_rollout = unitary_rollout(problem.trajectory, control_system)

    ax = f[i, 1] = Axis(f, xlabel = "Time", ylabel = "Unitary component", title="Robustness: $(i == 1 ? "No" : "Yes")")
    for (row, is_real) in zip(eachrow(Ũ⃗_rollout), operator_to_iso_vec(ones(2, 2)))
        lines!(ax, Δt .* (1:T), row, linewidth=5, linestyle = Bool(is_real) ? :solid : :dash)
    end
    println("Robustness: $(i == 1 ? "No" : "Yes")")
    println("\tInfidelity: ", 1 - unitary_fidelity(iso_vec_to_operator(Ũ⃗_rollout[:, end]), U_goal))
    println()
end
f

In [ ]:
# System with drift
f = Figure(resolution = (800, 600))
for (i, problem) in enumerate([prob, robust_prob])
    Ũ⃗_rollout = unitary_rollout(problem.trajectory, system)

    ax = f[i, 1] = Axis(f, xlabel = "Time", ylabel = "Unitary component", title="Robustness: $(i == 1 ? "No" : "Yes")")
    for (row, is_real) in zip(eachrow(Ũ⃗_rollout), operator_to_iso_vec(ones(2, 2)))
        lines!(ax, Δt .* (1:T), row, linewidth=5, linestyle = Bool(is_real) ? :solid : :dash)
    end
    println("Robustness: $(i == 1 ? "No" : "Yes")")
    println("\tInfidelity: ", 1 - unitary_fidelity(iso_vec_to_operator(Ũ⃗_rollout[:, end]), U_goal))
    println()
end


f